In [1]:
import os 
import pandas as pd
import numpy as np
from timeit import default_timer as timer
import matplotlib.pyplot as plt

import re
import glob

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [2]:
os.chdir('C:/Users/U/Documents/D3_Finance/quarterly_reports')

In [3]:
pd.options.display.max_rows = 40

In [4]:
meta_list = ['Fiscal Year', 'Quarter', 'Item Num', 'Item Name']

### CONSOLIDATED INCOME STATEMENT

In [5]:
### CONSOLIDATED INCOME STATEMENT

long_df = pd.DataFrame()
for pdf_file in glob.glob('*.pdf'): 
#for pdf_file in glob.glob('Q1FY17_Earnings_Release.pdf'): 
    #print('\n', pdf_file)
    quarter = int(pdf_file[1:2])
    fy = int(pdf_file[4:6])
    fyq = pdf_file[4:6] + 'Q' + pdf_file[1:2]
    if fy >= 10:
    #if (quarter >= 1) & (fy >= 16):
        if (quarter >= 2) & (fy >= 15):
            column_list = ['Quarterly', 'Quarterly Prev', 'Quarterly % Change', 'Cumulative', 'Cumlative Prev', 'Cumulative % Change']
        if (quarter == 1) & (fy >= 16):
            column_list = ['Quarterly', 'Quarterly Prev', 'Quarterly % Change']
        if (quarter >= 2) & (fy < 15):
            column_list = ['Quarterly', 'Quarterly Prev', 'Cumulative', 'Cumlative Prev']
        if (quarter == 1) & (fy < 16):
            column_list = ['Quarterly', 'Quarterly Prev']

        num_col = len(column_list)
        
        rsrcmgr = PDFResourceManager()
        rettxt = StringIO()
        laparams = LAParams()
        device = TextConverter(rsrcmgr, rettxt, codec='utf-8', laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        
        fp = open(pdf_file, 'rb')

        for page in PDFPage.get_pages(fp, pagenos=[1], maxpages=1, password=None,caching=True, check_extractable=True):
            interpreter.process_page(page)
        t = rettxt.getvalue()
        #print('First 400 chars:\n', t[0:400])
        #print('All text contents:\n', t)
        fp.close()
        
        init=0
        t = re.split("1  CONSOLIDATED INCOME STATEMENT", t)[-1]
        
        tl = re.split('\n',t)

        nl = []
        il = []

        for i in range(len(tl)-init):
            s = tl[init+i]
            if not re.match('20[0-1]\d', s):
                if s in ['n.m.', '-', '--']:
                    n = np.NaN
                    nl.append(n)
                else:
                    swoc = s.replace(',','')
                    #rn = re.match('\((\d+)\)', swoc)
                    rn = re.match('\((\d+\.?\d*)\)$', swoc)
                    if rn:
                        n = - float(rn.group(1))
                        nl.append(n)
                    else:
                        #r = re.match('(\d+)', swoc)
                        r = re.match('(\d+\.?\d*)$', swoc)
                        if r:
                            n = float(r.group(1))
                            nl.append(n)
                        else:
                            if s in ["US$'000"
                                         ,'Three months ended'
                                         ,'Half year ended'
                                         ,'Nine months ended'
                                         ,'Financial year ended'
                                         ,'% change'
                                         ,'\+ \/ \(\-\)'
                                         ,'31 March'
                                         ,'30 June'
                                         ,'30 September'
                                         ,'Expenses:'
                                         , 'Attributable to:'
                                     , 'Loss per share for loss attributable to'
                                         , 'equity holders of the Company'] or re.search('([A-Z][A-Z][A-Z])|(.*attributable to)', s):
                                pass
                            else:
                                r = re.match('(\- )?\(?[A-Z]+', swoc)
                                #r = re.match('[A-Z]+', swoc)
                                #r = True
                                if r:
                                    il.append(s)
        
        num_num = len(nl)
        num_row = num_num / num_col
        num_row_floor = num_num // num_col
        num_items = len(il)
        
        print('# of numbers catched:', num_num)
        print('# of rows catched:', num_row)
        print('First number catched:', nl[0])
        print('# of items catched: ', num_items)
        if num_row_floor != num_items:
            print('# of number and # of items do not match!!!!!!!!!')
        for i in range(num_items):
            item = il[i]
            print(i, '\t', item)
            
            item \
            = item\
            .replace('(loss)', '').replace('(Loss) profit', 'Profit').replace('(Loss) ', '')\
            .replace('(losses)', '').replace('(Losses)', ' ')\
            .replace('loss', 'profit').replace('Loss ', 'Profit ')\
            .replace(', net', '')\
            .replace('gains', 'gain')\
            .replace('profites', 'profit').replace('profits', 'profit')
            
            item \
            = re.sub('  +', ' ', item)
            
            item = item.strip()
            
            item = item\
            .replace('Income tax credit (expense)', 'Income tax expense')\
            .replace('Income tax credit', 'Income tax expense')\
            .replace('Income tax (expense) credit', 'Income tax expense')\
            
            il[i] = item
        
        ### Correct the order different from appearance in Adobe Acrobat Reader
        if (fy == 16) & (quarter in [2]):
            nl.insert(9*num_col, nl.pop(9*num_col+2)) ## (Loss) profit before income tax
            nl.insert(10*num_col, nl.pop(10*num_col+1)) ## Income tax expense
        if (fy == 16) & (quarter in [3]):
            nl.insert(10*num_col, nl.pop(10*num_col+2)) ## (Loss) profit before income tax
            nl.insert(11*num_col, nl.pop(11*num_col+1)) ## Income tax expense
        if (fy == 17) & (quarter in [2]):
            nl.insert(10*num_col-1, nl.pop(11*num_col-1)) ## (Loss) profit before income tax

        ### Correct the sign of COGS
        if ((quarter >= 2) & (fy < 16)) | (quarter == 1) & (fy < 12):
            nl.insert(1*num_col, - nl.pop(1*num_col)) ## negate COGS   
        
        n_arr = np.array(nl)

        n_2darr = n_arr[:num_row_floor*num_col].reshape(num_row_floor,num_col)
        
        df = pd.DataFrame(n_2darr, columns = column_list)
        print('df.shape[0]:',df.shape[0])
        df.index.name = 'Item Num'
        df.reset_index(inplace=True)
        #df['Item Name'] = 'To be filled' #il[:(df.shape[0])] 
        for i in range(min(num_items, num_row_floor)):
            df.loc[i, 'Item Name'] = il[i]
        df['Fiscal Year'] = fy
        df['Quarter'] = quarter
        df.set_index(meta_list, inplace=True)
        df.reset_index(inplace=True)

        display(df)
        long_df = long_df.append(df[meta_list+['Quarterly']], ignore_index=True)

        
item_num_df = long_df.groupby('Item Name')[['Item Num']].max().reset_index()
long2_df = long_df.drop(columns=['Item Num'], inplace=False).copy()
long2_df = pd.merge(long2_df, item_num_df, how = 'left', on = 'Item Name')

long2_df['FYQ'] = long2_df[['Fiscal Year', 'Quarter']].apply(lambda x: '{0:d}Q{1:d}'.format(x[0], x[1]), axis=1)
#long2_df

long2_df.set_index(['FYQ', 'Item Num', 'Item Name'], inplace=True)
wide_df = long2_df['Quarterly'].unstack(level=[1,2])

wide_df.columns = wide_df.columns.get_level_values('Item Name') ### Flatten the multiindex columns



device.close()
rettxt.close()

# of numbers catched: 34
# of rows catched: 17.0
First number catched: 72059.0
# of items catched:  17
0 	 Sales, net
1 	 Cost of goods sold
2 	 Gross profit 
3 	 Selling, general and administrative 
4 	 Research and development
5 	 Total expenses
6 	 Other  income
7 	 Other gains (losses), net
8 	 Share of profit (loss) of associated companies
9 	 Interest expense
10 	 Loss before income tax
11 	 Income tax expense
12 	 Net loss
13 	 Equity holders of the Company
14 	 Minority interests
15 	 - Basic (US$ per share)
16 	 - Diluted (US$ per share)
df.shape[0]: 17


,Fiscal Year,Quarter,Item Num,Item Name,Quarterly,Quarterly Prev
0,10,1,0,Sales,72059.00,141230.00
1,10,1,1,Cost of goods sold,-54230.00,110608.00
2,10,1,2,Gross profit,17829.00,30622.00
3,10,1,3,"Selling, general and administrative",-14803.00,-29873.00
4,10,1,4,Research and development,-14653.00,-17674.00
5,10,1,5,Total expenses,-29456.00,-47547.00
6,10,1,6,Other income,219.00,1793.00
7,10,1,7,Other gain,10522.00,-24402.00
8,10,1,8,Share of profit of associated companies,38.00,-938.00
9,10,1,9,Interest expense,-29.00,-77.00


# of numbers catched: 34
# of rows catched: 17.0
First number catched: 57227.0
# of items catched:  17
0 	 Sales, net
1 	 Cost of goods sold
2 	 Gross profit 
3 	 Selling, general and administrative 
4 	 Research and development
5 	 Total expenses
6 	 Other  income
7 	 Other gains (losses), net
8 	 Share of profit of associated companies
9 	 Interest expense
10 	 Loss before income tax
11 	 Income tax expense
12 	 Net loss
13 	 Equity holders of the Company
14 	 Non-controlling interests
15 	 - Basic (US$ per share)
16 	 - Diluted (US$ per share)
df.shape[0]: 17


,Fiscal Year,Quarter,Item Num,Item Name,Quarterly,Quarterly Prev
0,11,1,0,Sales,57227.00,72059.00
1,11,1,1,Cost of goods sold,-43192.00,54230.00
2,11,1,2,Gross profit,14035.00,17829.00
3,11,1,3,"Selling, general and administrative",-14172.00,-14803.00
4,11,1,4,Research and development,-16620.00,-14653.00
5,11,1,5,Total expenses,-30792.00,-29456.00
6,11,1,6,Other income,167.00,219.00
7,11,1,7,Other gain,13112.00,10522.00
8,11,1,8,Share of profit of associated companies,NaN,38.00
9,11,1,9,Interest expense,-20.00,-29.00


# of numbers catched: 34
# of rows catched: 17.0
First number catched: 50356.0
# of items catched:  17
0 	 Sales, net
1 	 Cost of goods sold
2 	 Gross profit 
3 	 Selling, general and administrative 
4 	 Research and development
5 	 Total expenses
6 	 Other  income
7 	 Other gains (losses), net
8 	 Share of losses of associated companies
9 	 Interest expense
10 	 Loss before income tax
11 	 Income tax expense
12 	 Net loss
13 	 Equity holders of the Company
14 	 Non-controlling interests
15 	 - Basic (US$ per share)
16 	 - Diluted (US$ per share)
df.shape[0]: 17


...

,Fiscal Year,Quarter,Item Num,Item Name,Quarterly,Quarterly Prev,Quarterly % Change,Cumulative,Cumlative Prev,Cumulative % Change
0,15,4,0,Sales,21157.00,23048.00,-8.00,99482.00,116332.0,-14.0
1,15,4,1,Cost of goods sold,-15237.00,18419.00,-17.00,71022.00,86250.0,-18.0
2,15,4,2,Gross profit,5920.00,4629.00,28.00,28460.00,30082.0,-5.0
3,15,4,3,"Selling, general and administrative",-10353.00,-7697.00,35.00,-36121.00,-34093.0,6.0
4,15,4,4,Research and development,-4867.00,-6398.00,-24.00,-19674.00,-25488.0,-23.0
5,15,4,5,Total expenses,-15220.00,-14095.00,8.00,-55795.00,-59581.0,-6.0
6,15,4,6,Other income,274.00,55.00,NaN,452.00,225.0,101.0
7,15,4,7,Other gain,9097.00,2311.00,NaN,9608.00,4002.0,-3491.0
8,15,4,8,Other profit,-819.00,NaN,-16848.00,-973.00,NaN,NaN
9,15,4,9,Profit before income tax,-3420.00,-7919.00,-57.00,-34123.00,-26245.0,30.0


# of numbers catched: 102
# of rows catched: 17.0
First number catched: 17102.0
# of items catched:  17
0 	 Sales, net
1 	 Cost of goods sold
2 	 Gross profit 
3 	 Selling, general and administrative 
4 	 Research and development
5 	 Total expenses
6 	 Other  income
7 	 Other gains
8 	 Other losses
9 	 Share of loss of associated companies
10 	 Profit (loss) before income tax
11 	 Income tax credit (expense)
12 	 Net profit (loss)
13 	 Equity holders of the Company
14 	 Non-controlling interests
15 	 - Basic (US$ per share)
16 	 - Diluted (US$ per share)
df.shape[0]: 17


,Fiscal Year,Quarter,Item Num,Item Name,Quarterly,Quarterly Prev,Quarterly % Change,Cumulative,Cumlative Prev,Cumulative % Change
0,16,4,0,Sales,17102.00,21157.00,-19.00,84574.00,99482.0,-15.0
1,16,4,1,Cost of goods sold,-12197.00,-15237.00,-20.00,-61605.00,-71022.0,-13.0
2,16,4,2,Gross profit,4905.00,5920.00,-17.00,22969.00,28460.0,-19.0
3,16,4,3,"Selling, general and administrative",-6087.00,-10353.00,-41.00,-36793.00,-36121.0,2.0
4,16,4,4,Research and development,-3267.00,-4867.00,-33.00,-15289.00,-19674.0,-22.0
5,16,4,5,Total expenses,-9354.00,-15220.00,-39.00,-52082.00,-55795.0,-7.0
6,16,4,6,Other income,94.00,274.00,-66.00,354.00,452.0,-22.0
7,16,4,7,Other gain,5298.00,9097.00,-42.00,41538.00,9608.0,332.0
8,16,4,8,Other profit,-302.00,-3491.00,-91.00,-1370.00,-16848.0,-92.0
9,16,4,9,Share of profit of associated companies,-377.00,NaN,NaN,-724.00,NaN,NaN


# of numbers catched: 96
# of rows catched: 16.0
First number catched: 14483.0
# of items catched:  16
0 	 Sales, net
1 	 Cost of goods sold
2 	 Gross profit 
3 	 Selling, general and administrative 
4 	 Research and development
5 	 Total expenses
6 	 Other  income
7 	 Other gains (losses)
8 	 Share of gain (loss) of associated companies
9 	 (Loss) profit before income tax
10 	 Income tax credit (expense)
11 	 Net (loss) profit
12 	 Equity holders of the Company
13 	 Non-controlling interests
14 	 - Basic (US$ per share)
15 	 - Diluted (US$ per share)
df.shape[0]: 16


,Fiscal Year,Quarter,Item Num,Item Name,Quarterly,Quarterly Prev,Quarterly % Change,Cumulative,Cumlative Prev,Cumulative % Change
0,17,4,0,Sales,14483.00,17102.00,-15.00,69895.00,84574.0,-17.0
1,17,4,1,Cost of goods sold,-10410.00,-12197.00,-15.00,-50300.00,-61605.0,-18.0
2,17,4,2,Gross profit,4073.00,4905.00,-17.00,19595.00,22969.0,-15.0
3,17,4,3,"Selling, general and administrative",-8160.00,-6087.00,34.00,-30404.00,-36793.0,-17.0
4,17,4,4,Research and development,-2878.00,-3267.00,-12.00,-12131.00,-15289.0,-21.0
5,17,4,5,Total expenses,-11038.00,-9354.00,18.00,-42535.00,-52082.0,-18.0
6,17,4,6,Other income,129.00,94.00,37.00,430.00,354.0,21.0
7,17,4,7,Other gain,874.00,4996.00,-83.00,-25.00,40168.0,NaN
8,17,4,8,Share of gain of associated companies,164.00,-377.00,NaN,-326.00,-724.0,-55.0
9,17,4,9,Profit before income tax,-5798.00,264.00,NaN,-22861.00,10685.0,NaN


In [6]:
wide_df['Interest expense'].fillna(value=0, inplace=True)
wide_df['EBIT'] = wide_df[['Interest expense', 'Profit before income tax']].apply(lambda x: (x[1] - x[0]), axis=1)
### Calculate ratios
wide_df['Gross profit margin'] = wide_df[['Sales', 'Gross profit']].apply(lambda x: (x[1] / x[0]), axis=1)
wide_df['EBIT margin'] = wide_df[['Sales', 'EBIT']].apply(lambda x: (x[1] / x[0]), axis=1)
wide_df['Profit before income tax margin']\
= wide_df[['Sales', 'Profit before income tax']].apply(lambda x: (x[1] / x[0]), axis=1)
wide_df['Net profit margin'] = wide_df[['Sales', 'Net profit']].apply(lambda x: (x[1] / x[0]), axis=1)
#wide_df['Operating cash flow'] = wide_df[['Gross profit', 'Income tax expense']].apply(lambda x: (x[0] + x[1]), axis=1)
wide_df['Reciprocal of times interest earned']\
= wide_df[['Interest expense', 'EBIT']].apply(lambda x: ( - x[0] / x[1]), axis=1)
wide_df['Reciprocal of cash coverage']\
= wide_df[['Gross profit', 'Selling, general and administrative', 'Interest expense']].apply(lambda x: (x[2] / (x[0] + x[1])), axis=1)

### Just for checking the calculation is correct
wide_df['Gross profit 0 check']\
= wide_df[['Sales', 'Cost of goods sold', 'Gross profit']].apply(lambda x: (x[0] + x[1] - x[2]), axis=1)
wide_df['Net profit 0 check']\
= wide_df[['Profit before income tax', 'Income tax expense', 'Net profit']].apply(lambda x: (x[0] + x[1] - x[2]), axis=1)

wide_df['Fiscal Year'] = wide_df.index.str[0:2]
wide_df.loc[:,'Quarter'] = wide_df.index.str[3:4]

wide_df.to_csv('IS.csv')

IS_ratios_df = wide_df[['Fiscal Year', 'Quarter', 'Sales', 'Cost of goods sold', 'Gross profit', 'Total expenses', 'EBIT', 'Profit before income tax', 'Income tax expense', 'Net profit'
                    , 'Gross profit margin', 'EBIT margin', 'Profit before income tax margin', 'Net profit margin', 'Reciprocal of times interest earned', 'Reciprocal of cash coverage']]
IS_ratios_df.to_csv('IS_Ratios.csv')

### BALANCE SHEETS

In [7]:
def swap_elements_in_list(target_list, i_insert, i_start, i_end):
    r = target_list[i_start:i_end]
    target_list[i_start:i_end] = []
    target_list[i_insert:i_insert] = r
    return target_list

In [8]:
### BALANCE SHEETS

long_df = pd.DataFrame()
for pdf_file in glob.glob('*.pdf'): 
#for pdf_file in glob.glob('Q4FY10_Earnings_Release.pdf'): 
    print('\n', pdf_file)
    quarter = int(pdf_file[1:2])
    fy = int(pdf_file[4:6])
    fyq = pdf_file[4:6] + 'Q' + pdf_file[1:2]
    if fy >= 10:
        column_list = ['Quarterly', 'Quarterly Prev', 'Company', 'Company Prev']
        num_col = len(column_list)
        
        rsrcmgr = PDFResourceManager()
        rettxt = StringIO()
        laparams = LAParams()
        device = TextConverter(rsrcmgr, rettxt, codec='utf-8', laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        
        fp = open(pdf_file, 'rb')
        if (fy in [10, 11, 12, 13]) & (quarter == 4):
            pageno = 4
        else:
            pageno = 3
        for page in PDFPage.get_pages(fp, pagenos=[pageno], maxpages=1, password=None,caching=True, check_extractable=True):
            interpreter.process_page(page)
        t = rettxt.getvalue()
        #print('All text contents:\n', t)
        fp.close()
        
        init=0
        t = re.split("3 BALANCE SHEETS", t)[-1]
        
        tl = re.split('\n',t)

        nl = []
        il = []

        for i in range(len(tl)-init):
            s = tl[init+i]
            if not re.match('20[0-1]\d', s):
                if s in ['n.m.']:
                    n = np.NaN
                    nl.append(n)
                    continue
                if s in ['-', '--']:
                    n = 0
                    nl.append(n)
                    continue
                s = s.strip()
                swoc = s.replace(',','')
                #rn = re.match('\((\d+)\)', swoc)
                rn = re.match('\((\d+\.?\d*)\)$', swoc)
                if rn:
                    n = - float(rn.group(1))
                    nl.append(n)
                else:
                    #r = re.match('(\d+)', swoc)
                    r = re.match('(\d+\.?\d*)$', swoc)
                    if r:
                        n = float(r.group(1))
                        nl.append(n)
                    else:
                        if s in ['Group', 'GROUP'
                                 ,'Company', 'COMPANY'
                                 , "US$'000"
                                 ,'Three months ended'
                                 ,'Half year ended'
                                 ,'Nine months ended'
                                 ,'Financial year ended'
                                 ,'31 March'
                                 ,'30 June'
                                 ,'30 September'
                                 ,'ASSETS'
                                 ,'LIABILITIES'
                                 , 'LIABILITIES AND EQUITY'
                                 ,'EQUITY'] :
                            pass
                        else: 
                            r = re.match('([A-Za-z \-]*:)', swoc)
                            if r:
                                cat = r.group(1)
                            if s in ['Amounts due from subsidiaries', 'Amounts due to subsidiaries']:
                                il.append(cat+' '+s)
                                continue
                            if s == 'Current assets:':
                                continue
                            if s == 'Non-current assets:':
                                il.append('Current assets')
                                continue
                            if s == 'Financial assets, available-for-sale':
                                il.append('Available-for-sale financial assets')
                                continue
                            if s == 'Total assets':
                                il.append('Non-current assets')
                                il.append(s)
                                continue
                            if s == 'Current liabilities:':
                                continue
                            if s == 'Non-current liabilities:':
                                il.append('Current liabilities')
                                continue
                            if s == 'Total liabilities':
                                il.append('Non-current liabilities')
                                il.append(s)
                                continue  
                            if s == 'NET ASSETS':
                                il.append('Total equity')
                                continue
                            if s == 'Equity:':
                                continue                                
                            if s in ['Non-controlling interests', 'Minority interests']:
                                il.append('Equity excluding non-controlling interests')
                                il.append('Non-controlling interests')
                                continue
                            if s == 'Total equity':
                                if 'Total equity' in il:
                                    il.append('Total equity_')
                                else:
                                    il.append(s)
                                continue
                            if s == 'Total liabilities and equity':
                                il.append(s)
                                continue
                            r = re.match('(\- )?\(?[A-Z]+', swoc)
                            if r:
                                il.append(s)
        
        if (fy == 18) & (quarter == 1):
            ## correct missing '-'
            nl.insert(nl.index(27478), 0) 
        
        if (fyq in ['14Q1', '14Q3', '14Q4'
                    , '15Q1'
                    , '16Q2', '16Q3', '16Q4']):
            ## Correct order of Non-current assets
            nl = swap_elements_in_list(nl
            , il.index('Current assets')*4 - (il.index('Non-current assets') - il.index('Current assets')) * 2
            , il.index('Current assets')*4
            , il.index('Current assets')*4 + (il.index('Current assets') - (-1)) * 2)
        
        if fyq in ['10Q4', '11Q1']:
            ## correct irregular non-currrent assets
            nl = swap_elements_in_list(nl, 12, 28, 34) 
            nl = swap_elements_in_list(nl, 18, 42, 48)
            
        
        if (fyq in ['11Q4', '12Q1', '12Q2', '12Q3']) or (fy in [13, 14, 15, 16, 17]):
            ## Correct order of Non-current liabilities
            nl = swap_elements_in_list(nl
            , il.index('Current liabilities')*4 - (il.index('Non-current liabilities') - il.index('Current liabilities')) * 2
            , il.index('Current liabilities')*4
            , il.index('Current liabilities')*4 + (il.index('Current liabilities') - il.index('Total assets')) * 2)
        
        if fyq == '10Q4':
            nl = swap_elements_in_list(nl, 82, 91, 97) ## correct irregular non-currrent liabilities
            
        if fyq in ['10Q3', '10Q4']:
            ## correct order by swapping irregular Trade payable and Current portion of long term obligations
            nl[64], nl[68] = nl[68], nl[64]        

        print('\nnl:\n')
        for i in range(len(nl)):
            print(i, nl[i])
            
        n_arr = np.array(nl)
                                         
        num_num = len(nl)
        num_row = num_num / num_col
        num_row_floor = num_num // num_col
        num_items = len(il)
        
        print('# of numbers catched:', num_num)
        print('# of rows catched:', num_row)
        print('First number catched:', nl[0])
        print('# of items catched: ', num_items)
        if num_row_floor != num_items:
            print('# of number and # of items do not match!!!!!!!!!')
        for i in range(num_items):
            item = il[i]
            print(i, '\t', item)

            il[i] = item
        
        #print(il)
               
        cat_item_list = \
        ['Current assets'
         , 'Non-current assets'
         , 'Total assets'
         , 'Current liabilities'
         , 'Non-current liabilities'
         , 'Total liabilities'
         , 'Total equity'
         , 'Total equity_'
         , 'Total liabilities and equity']
        
        if (fyq in ['10Q1', '10Q2','10Q3', '13Q3', '13Q4', '14Q2', '15Q2', '15Q3', '15Q4', '16Q1']):
            cat_item_list.pop(0)
        print('cat_item_list:', cat_item_list)
        
        #cat_index_list = [(il.index(item) + 1)*num_col for item in cat_item_list]
        cat_index_list  = []
        for item in cat_item_list:
            if item in il:
                cat_index_list.append((il.index(item) + 1)*num_col)
        print('cat_index_list:', cat_index_list )

        for i in range(len(cat_index_list)):  
            if i == 0:
                ci_prev = 0
            ci = cat_index_list[i]
            n_sub_arr = n_arr[ci_prev:ci] #nl_sub = nl[ci_prev:ci]          
            #print('\nnl_sub: \n', nl_sub)
            ci_prev = ci
            
            num_sub = len(n_sub_arr) #num_sub = len(nl_sub)
            ##n_sub_arr = np.array(nl_sub)
            n_sub_2darr = n_sub_arr[0:(num_col * (num_sub // num_col))].reshape(num_col, (num_sub // num_col))
            #print('\nn_sub_2darr: \n', n_sub_2darr)
            n_sub_t_2darr = n_sub_2darr.transpose()
            #print('\nn_sub_t_2darr: \n', n_sub_t_2darr)
            n_sub_1darr = n_sub_t_2darr.reshape((num_col * (num_sub // num_col)))
            n_sub_s_2darr = n_sub_1darr.reshape((num_sub // num_col), num_col)
            print('\niteration', i, ': n_sub_s_2darr \n', n_sub_s_2darr)
            if i == 0:
                n_2darr = n_sub_s_2darr
            else:
                n_2darr = np.vstack([n_2darr, n_sub_s_2darr])

        if fy == 13:
            n_2darr[[2,3]] = n_2darr[[3,2]] ### correct Inventories 

        if fyq == '12Q4':
            n_2darr[12, 0], n_2darr[13, 0] = n_2darr[13, 0], n_2darr[12, 0]
            nl.insert(12*4, nl.pop(13*4)) ### Swap Property and equipment with Intangible assets
      
        df = pd.DataFrame(n_2darr, columns = column_list)
        print('\ndf.shape[0]:',df.shape[0])
        df.index.name = 'Item Num'
        df.reset_index(inplace=True)
        #df['Item Name'] = 'To be filled' #il[:(df.shape[0])] 
        for i in range(min(num_items, num_row_floor)):
            df.loc[i, 'Item Name'] = il[i]
        df['Fiscal Year'] = fy
        df['Quarter'] = quarter
        df.set_index(meta_list, inplace=True)
        df.reset_index(inplace=True)

        display(df)
        long_df = long_df.append(df[meta_list+['Quarterly']], ignore_index=True)
        
item_num_df = long_df.groupby('Item Name')[['Item Num']].max().reset_index()
long2_df = long_df.drop(columns=['Item Num'], inplace=False).copy()
long2_df = pd.merge(long2_df, item_num_df, how = 'left', on = 'Item Name')

long2_df['FYQ'] = long2_df[['Fiscal Year', 'Quarter']].apply(lambda x: '{0:d}Q{1:d}'.format(x[0], x[1]), axis=1)
#long2_df

long2_df.set_index(['FYQ', 'Item Num', 'Item Name'], inplace=True)
print('\nExtracted data in long (stacked) format:')
display(long2_df)
print('\nDuplicated: ')
dup_df = long2_df[long2_df.index.duplicated(keep=False)]
display(dup_df.reset_index()['FYQ'].unique())

wide_df = long2_df['Quarterly'].unstack(level=[1,2])

wide_df.columns = wide_df.columns.get_level_values('Item Name') ### Flatten the multiindex columns

device.close()
rettxt.close()


 Q1FY07_Earnings_Release.pdf

 Q1FY08_Earnings_Release.pdf

 Q1FY09_Earnings_Release.pdf

 Q1FY10_Earnings_Release.pdf

nl:

0 253066.0
1 40247.0
2 0
3 40702.0
4 20571.0
5 354586.0
6 31609.0
7 1296.0
8 0
9 0
10 0
11 33341.0
12 6628.0
13 29578.0
14 102452.0
15 250551.0
16 42365.0
17 0
18 37600.0
19 21448.0
20 351964.0
21 1372.0
22 27753.0
23 977.0
24 0
25 0
26 33944.0
27 6719.0
28 31693.0
29 102458.0
30 229365.0
31 4651.0
32 25294.0
33 22302.0
34 4435.0
35 286047.0
36 218472.0
37 6017.0
38 43379.0
39 18104.0
40 4934.0
41 290906.0
42 188988.0
43 0
44 0
45 0
46 0
47 818.0
48 237279.0
49 45390.0
50 2083.0
51 186395.0
52 0
53 0
54 0
55 0
56 768.0
57 234613.0
58 45390.0
59 2060.0
60 457038.0
61 454422.0
62 523326.0
63 525519.0
64 32635.0
65 0
66 67184.0
67 2185.0
68 9.0
69 102013.0
70 30296.0
71 0
72 70014.0
73 2203.0
74 10.0
75 102523.0
76 18046.0
77 16169.0
78 29356.0
79 2000.0
80 8.0
81 65579.0
82 18608.0
83 17174.0
84 28852.0
85 2000.0
86 10.0
87 66644.0
88 0
89 29510.0
90 29510.0
91 0


,Fiscal Year,Quarter,Item Num,Item Name,Quarterly,Quarterly Prev,Company,Company Prev
0,10,1,0,Cash and cash equivalents,253066.0,250551.0,229365.0,0.0
1,10,1,1,Trade receivables,40247.0,42365.0,4651.0,0.0
2,10,1,2,Current assets: Amounts due from subsidiaries,0.0,0.0,25294.0,818.0
3,10,1,3,Inventories,40702.0,37600.0,22302.0,237279.0
4,10,1,4,Other current assets,20571.0,21448.0,4435.0,45390.0
5,10,1,5,Current assets,354586.0,351964.0,286047.0,2083.0
6,10,1,6,Available-for-sale financial assets,31609.0,1372.0,218472.0,186395.0
7,10,1,7,Other non-current receivables,1296.0,27753.0,6017.0,0.0
8,10,1,8,Non-current assets: Amounts due from subsidiaries,0.0,977.0,43379.0,0.0
9,10,1,9,Investments in associated companies,0.0,0.0,18104.0,0.0



 Q1FY11_Earnings_Release.pdf

nl:

0 200965.0
1 32286.0
2 0
3 45650.0
4 25207.0
5 304108.0
6 214025.0
7 28999.0
8 0
9 46120.0
10 20990.0
11 310134.0
12 178537.0
13 3579.0
14 43100.0
15 24506.0
16 4649.0
17 254371.0
18 197803.0
19 4355.0
20 28880.0
21 25502.0
22 4306.0
23 260846.0
24 39174.0
25 582.0
26 0
27 0
28 27994.0
29 6214.0
30 20618.0
31 94582.0
32 33895.0
33 582.0
34 0
35 0
36 29411.0
37 6324.0
38 22205.0
39 92417.0
40 186162.0
41 45390.0
42 1233.0
43 0
44 0
45 0
46 953.0
47 233738.0
48 184282.0
49 45390.0
50 1394.0
51 0
52 0
53 0
54 998.0
55 232064.0
56 398690.0
57 402551.0
58 488109.0
59 492910.0
60 19815.0
61 0
62 63208.0
63 2233.0
64 85256.0
65 24089.0
66 0
67 65249.0
68 2235.0
69 91573.0
70 13525.0
71 25587.0
72 29765.0
73 1951.0
74 70828.0
75 18218.0
76 23238.0
77 30777.0
78 1951.0
79 74184.0
80 0
81 21202.0
82 21202.0
83 0
84 21202.0
85 21202.0
86 21047.0
87 0
88 21047.0
89 21044.0
90 0
91 21044.0
92 106458.0
93 112775.0
94 91875.0
95 95228.0
96 292232.0
97 289776.0
98 3

,Fiscal Year,Quarter,Item Num,Item Name,Quarterly,Quarterly Prev,Company,Company Prev
0,11,1,0,Cash and cash equivalents,200965.0,214025.0,178537.0,197803.0
1,11,1,1,Trade receivables,32286.0,28999.0,3579.0,4355.0
2,11,1,2,Current assets: Amounts due from subsidiaries,0.0,0.0,43100.0,28880.0
3,11,1,3,Inventories,45650.0,46120.0,24506.0,25502.0
4,11,1,4,Other current assets,25207.0,20990.0,4649.0,4306.0
5,11,1,5,Current assets,304108.0,310134.0,254371.0,260846.0
6,11,1,6,Available-for-sale financial assets,39174.0,33895.0,186162.0,184282.0
7,11,1,7,Other non-current receivables,582.0,582.0,45390.0,45390.0
8,11,1,8,Non-current assets: Amounts due from subsidiaries,0.0,0.0,1233.0,1394.0
9,11,1,9,Investments in subsidiaries,0.0,0.0,0.0,0.0



 Q1FY12_Earnings_Release.pdf

nl:

0 146202.0
1 29378.0
2 0
3 38243.0
4 19341.0
5 233164.0
6 162958.0
7 27508.0
8 0
9 44089.0
10 22642.0
11 257197.0
12 131514.0
13 8773.0
14 40554.0
15 18103.0
16 4482.0
17 203426.0
18 148693.0
19 8059.0
20 37424.0
21 23425.0
22 5966.0
23 223567.0
24 27126.0
25 283.0
26 0
27 0
28 0
29 28215.0
30 5742.0
31 12399.0
32 73765.0
33 36476.0
34 460.0
35 0
36 322.0
37 0
38 26088.0
39 5862.0
40 15600.0
41 84808.0
42 196703.0
43 0
44 0
45 0
46 30790.0
47 805.0
48 0
49 1688.0
50 229986.0
51 194391.0
52 0
53 0
54 0
55 30790.0
56 920.0
57 0
58 1422.0
59 227523.0
60 306929.0
61 342005.0
62 433412.0
63 451090.0
64 23282.0
65 0
66 58941.0
67 746.0
68 82969.0
69 20629.0
70 0
71 60662.0
72 790.0
73 82081.0
74 13216.0
75 28834.0
76 27566.0
77 0
78 69616.0
79 15203.0
80 29312.0
81 28668.0
82 0
83 73183.0
84 0
85 17902.0
86 17902.0
87 0
88 17902.0
89 17902.0
90 21146.0
91 0
92 21146.0
93 21149.0
94 0
95 21149.0
96 100871.0
97 99983.0
98 90762.0
99 94332.0
100 206058.0
101 

...

,Fiscal Year,Quarter,Item Num,Item Name,Quarterly,Quarterly Prev,Company,Company Prev
0,17,4,0,Cash and cash equivalents,75282.0,94738.0,69892.0,88362.0
1,17,4,1,Trade receivables,5850.0,7570.0,193.0,245.0
2,17,4,2,Current assets: Amounts due from subsidiaries,0.0,0.0,36277.0,43979.0
3,17,4,3,Inventories,20413.0,25052.0,2489.0,5153.0
4,17,4,4,Other current assets,1422.0,1797.0,597.0,633.0
5,17,4,5,Current assets,102967.0,129157.0,109448.0,138372.0
6,17,4,6,Available-for-sale financial assets,9622.0,11960.0,0.0,0.0
7,17,4,7,Non-current assets: Amounts due from subsidiaries,0.0,0.0,21323.0,14923.0
8,17,4,8,Investments in subsidiaries,0.0,0.0,14665.0,25353.0
9,17,4,9,Property and equipment,866.0,870.0,143.0,131.0



Extracted data in long (stacked) format:


Fiscal Year  \
FYQ  Item Num Item Name                                                         
10Q1 0        Cash and cash equivalents                                    10   
     1        Trade receivables                                            10   
     2        Current assets: Amounts due from subsidiaries                10   
     3        Inventories                                                  10   
     5        Other current assets                                         10   
     6        Current assets                                               10   
     7        Available-for-sale financial assets                          10   
     8        Other non-current receivables                                10   
     9        Non-current assets: Amounts due from subsidiaries            10   
     10       Investments in associated companies                          10   
     11       Investments in subsidiaries                                  10   
     12       Property and equipment                                       10   
     13       Intangible assets                                            10   
     14       Other non-current assets                                     10   
     15       Non-current assets                                           10   
     16       Total assets                                                 10   
     17       Trade payables                                               10   
     18       Current liabilities: Amounts due to subsidiaries             10   
     19       Accrued liabilities and provisions                           10   
     20       Current income tax liabilities                               10   
...                                                                       ...   
17Q4 15       Non-current assets                                           17   
     16       Total assets                                                 17   
     17       Trade payables                                               17   
     18       Current liabilities: Amounts due to subsidiaries             17   
     19       Accrued liabilities and provisions                           17   
     20       Current income tax liabilities                               17   
     21       Current liabilities                                          17   
     22       Non-current liabilities: Amounts due to subsidi...           17   
     23       Deferred income tax liabilities                              17   
     24       Non-current liabilities                                      17   
     25       Total liabilities                                            17   
     33       Total equity                                                 17   
     27       Share capital                                                17   
     28       Treasury shares                                              17   
     29       Fair value reserve                                           17   
     30       Other reserves                                               17   
     27       Accumulated losses                                           17   
     32       Equity excluding non-controlling interests                   17   
     33       Non-controlling interests                                    17   
     34       Total equity_                                                17   

                                                                  Quarter  \
FYQ  Item Num Item Name                                                     
10Q1 0        Cash and cash equivalents                                 1   
     1        Trade receivables                                         1   
     2        Current assets: Amounts due from subsidiaries             1   
     3        Inventories                                               1   
     5        Other current assets                                      1   
     6        Current assets               


Duplicated: 


array([], dtype=float64)

In [9]:

wide_df['Net working capital'] = wide_df[['Current assets', 'Current liabilities']].apply(lambda x: (x[0] - x[1]), axis=1)
### Calculate ratios
wide_df['Current ratio'] = wide_df[['Current liabilities', 'Current assets']].apply(lambda x: (x[1] / x[0]), axis=1)
wide_df['Quick ratio'] = wide_df[['Current liabilities', 'Current assets', 'Inventories']].apply(lambda x: ((x[1] - x[2]) / x[0]), axis=1)
wide_df['Cash ratio'] = wide_df[['Cash and cash equivalents', 'Current liabilities']].apply(lambda x: (x[0] / x[1]), axis=1)
wide_df['NWC to assets ratio'] = wide_df[['Total assets', 'Net working capital']].apply(lambda x: (x[1] / x[0]), axis=1)

wide_df['Debt ratio'] = wide_df[['Total assets', 'Total liabilities']].apply(lambda x: (x[1] / x[0]), axis=1)
wide_df['Debt to equity ratio'] = wide_df[['Total liabilities', 'Total equity']].apply(lambda x: (x[0] / x[1]), axis=1)
wide_df['Equity multiplier'] = wide_df[['Debt to equity ratio']].apply(lambda x: (1 + x[0]), axis=1)
wide_df['Long-term debt ratio'] = wide_df[['Non-current liabilities', 'Total equity']].apply(lambda x: (x[0] / x[1]), axis=1)

### Just for checking the calculation is correct
wide_df['Assets less liabilities less equity 0 check']\
= wide_df[['Total assets', 'Total liabilities', 'Total equity']].apply(lambda x: (x[0] - x[1] - x[2]), axis=1)
wide_df['Assets less current assets less non-current assets 0 check']\
= wide_df[['Total assets', 'Current assets', 'Non-current assets']].apply(lambda x: (x[0] - x[1] - x[2]), axis=1)
wide_df['Liabilities less current liabilities less non-current liabilities 0 check']\
= wide_df[['Total liabilities', 'Current liabilities', 'Non-current liabilities']].apply(lambda x: (x[0] - x[1] - x[2]), axis=1)

wide_df['Fiscal Year'] = wide_df.index.str[0:2]
wide_df.loc[:,'Quarter'] = wide_df.index.str[3:4]

wide_df.to_csv('BS.csv')

BS_ratios_df = wide_df[['Fiscal Year', 'Quarter', 'Cash and cash equivalents', 'Trade receivables', 'Inventories'
                    , 'Other current assets', 'Current assets', 'Available-for-sale financial assets'
                    , 'Property and equipment', 'Other non-current assets', 'Non-current assets', 'Total assets'
                    , 'Trade payables', 'Current income tax liabilities', 'Current liabilities'
                    , 'Deferred income tax liabilities', 'Non-current liabilities', 'Total liabilities', 'Total equity'
                    , 'Net working capital'
                    , 'Current ratio', 'Quick ratio', 'Cash ratio', 'NWC to assets ratio'
                    , 'Debt ratio', 'Debt to equity ratio', 'Equity multiplier', 'Long-term debt ratio' ]]
#display(BS_ratios_df)
BS_ratios_df.to_csv('BS_Ratios.csv')

In [10]:
merged_df = pd.merge(IS_ratios_df, BS_ratios_df, on = ['FYQ', 'Fiscal Year', 'Quarter'])

In [11]:
### Calculate ratios
merged_df['Days interval measure'] = merged_df[['Cost of goods sold', 'Total expenses', 'Current assets']]\
.apply(lambda x: (x[2] / ((- x[0] - x[1]) / (365/4))), axis=1)

merged_df['Inventory turnover'] = merged_df[['Cost of goods sold', 'Inventories']].apply(lambda x: ( - x[0] / x[1]), axis=1)
merged_df['Days sales in inventory'] = merged_df[['Inventory turnover']].apply(lambda x: ((365/4) / x[0]), axis=1)

merged_df['Receivables turnover'] = merged_df[['Sales', 'Trade receivables']].apply(lambda x: (x[0] / x[1]), axis=1)
merged_df['Days sales in receivable'] = merged_df[['Receivables turnover']].apply(lambda x: ((365/4) / x[0]), axis=1)

merged_df['Total asset turnover'] = merged_df[['Sales', 'Total assets']].apply(lambda x: (x[0] / x[1]), axis=1)
merged_df['Net working capital turnover']\
       = merged_df[['Sales', 'Net working capital']].apply(lambda x: (x[0] / x[1]), axis=1)
#merged_df['Fixed asset turnover'] = merged_df[['Sales', 'Propoerty and equipment', 'Depreciation]].apply(lambda x: (x[0] / (x[1] - x[2])), axis=1)

merged_df['Return on assets'] = merged_df[['Net profit', 'Total assets']].apply(lambda x: (x[0] / x[1]), axis=1)
merged_df['Return on equity'] = merged_df[['Net profit', 'Total equity']].apply(lambda x: (x[0] / x[1]), axis=1)
merged_df['Return on capital employed'] \
= merged_df[['EBIT', 'Total equity', 'Non-current liabilities']].apply(lambda x: (x[0] / (x[1] + x[2])), axis=1)
merged_df['Capital employed turnover'] \
= merged_df[['Sales', 'Total equity', 'Non-current liabilities']].apply(lambda x: (x[0] / (x[1] + x[2])), axis=1)


## Stock price data

In [12]:
price_df = pd.read_csv('C76.SI.csv', parse_dates=[0])
#price_df

In [13]:
#price_q_df = price_df.set_index('Date').groupby(pd.TimeGrouper('Q')).last().reset_index()
price_q_df = price_df.set_index('Date').groupby(pd.TimeGrouper('Q')).mean().reset_index()
#price_q_df

C:\Users\U\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  


In [14]:
def increment_month_last_day(date_series, number_months_to_add_series):  
    return pd.Series((date_series.values.astype("M8[M]")   
                + (number_months_to_add_series + 1) * np.timedelta64(1, "M")  
                ).astype("M8[D]") - np.timedelta64(1, "D"))
def fyq(date_series):
    s = increment_month_last_day(date_series, 6)
    return (s.dt.strftime('%y') + 'Q' + (s.dt.strftime('%m').astype(int)//3).astype(str))

In [15]:
price_q_df['FYQ'] = fyq(price_q_df['Date'])
price_q_df['Price'] = price_q_df['Close']
#price_q_df

In [16]:
merged3_df = pd.merge(merged_df, price_q_df[['FYQ', 'Price']], on = ['FYQ'])
merged3_df

,FYQ,Fiscal Year,Quarter,Sales,Cost of goods sold,Gross profit,Total expenses,EBIT,Profit before income tax,Income tax expense,...,Days sales in inventory,Receivables turnover,Days sales in receivable,Total asset turnover,Net working capital turnover,Return on assets,Return on equity,Return on capital employed,Capital employed turnover,Price
0,10Q1,10,1,72059.0,-54230.0,17829.0,-29456.0,-848.0,-877.0,-166.0,...,68.487138,1.790419,50.965719,0.157665,0.285300,-0.002282,-0.003204,-0.002389,0.202969,5.329385
1,10Q2,10,2,80713.0,-60174.0,20539.0,-42534.0,-35136.0,-35137.0,-351.0,...,55.918565,1.930841,47.259209,0.181386,0.331954,-0.079752,-0.111494,-0.101609,0.233412,5.979508
2,10Q3,10,3,63291.0,-46948.0,16343.0,-45666.0,-47449.0,-47483.0,-115.0,...,68.260629,2.027713,45.001442,0.149299,0.275617,-0.112281,-0.155396,-0.142146,0.189605,5.721148
3,10Q4,10,4,59244.0,-45020.0,14224.0,-33683.0,-15199.0,-15222.0,1258.0,...,93.479565,2.042967,44.665430,0.147171,0.271064,-0.034689,-0.048189,-0.048875,0.190509,4.668095
4,11Q1,11,1,57227.0,-43192.0,14035.0,-30792.0,-3478.0,-3498.0,-116.0,...,96.442918,1.772502,51.480901,0.143538,0.261487,-0.009065,-0.012367,-0.011096,0.182581,4.053750
5,11Q2,11,2,69289.0,-51529.0,17760.0,-31524.0,-11182.0,-11203.0,351.0,...,84.779323,2.205602,41.371917,0.178036,0.339456,-0.027884,-0.038954,-0.037300,0.231127,4.185469
6,11Q3,11,3,57154.0,-43427.0,13727.0,-33245.0,-12709.0,-12729.0,-247.0,...,101.119465,1.876609,48.624943,0.155655,0.295899,-0.035339,-0.049774,-0.045083,0.202746,3.742903
7,11Q4,11,4,47355.0,-43816.0,3539.0,-32396.0,-24462.0,-24473.0,4772.0,...,91.818542,1.721499,53.006124,0.138463,0.270421,-0.057604,-0.081402,-0.094112,0.182188,3.296452
8,12Q1,12,1,50356.0,-41450.0,8906.0,-26221.0,-29455.0,-29466.0,-101.0,...,84.189958,1.714072,53.235811,0.164064,0.335271,-0.096332,-0.143489,-0.131519,0.224844,2.736875
9,12Q2,12,2,59547.0,-47429.0,12118.0,-21336.0,-33777.0,-33787.0,-77.0,...,62.008212,1.943250,46.957424,0.215943,0.459269,-0.122806,-0.198914,-0.179525,0.316494,2.504839


In [17]:
### Calculate ratios
merged3_df['EPS'] = merged3_df[['Net profit']].apply(lambda x: (x[0] / 70330000), axis=1) ## of shares outstanding = 70.33M

merged3_df['PE ratio'] = merged3_df[['Price', 'EPS']]\
.apply(lambda x: (x[0] / x[1] ), axis=1)

In [18]:
merged3_df.to_csv('ISBS_Ratios.csv')

In [19]:
merged3_df.columns

Index(['FYQ', 'Fiscal Year', 'Quarter', 'Sales', 'Cost of goods sold',
       'Gross profit', 'Total expenses', 'EBIT', 'Profit before income tax',
       'Income tax expense', 'Net profit', 'Gross profit margin',
       'EBIT margin', 'Profit before income tax margin', 'Net profit margin',
       'Reciprocal of times interest earned', 'Reciprocal of cash coverage',
       'Cash and cash equivalents', 'Trade receivables', 'Inventories',
       'Other current assets', 'Current assets',
       'Available-for-sale financial assets', 'Property and equipment',
       'Other non-current assets', 'Non-current assets', 'Total assets',
       'Trade payables', 'Current income tax liabilities',
       'Current liabilities', 'Deferred income tax liabilities',
       'Non-current liabilities', 'Total liabilities', 'Total equity',
       'Net working capital', 'Current ratio', 'Quick ratio', 'Cash ratio',
       'NWC to assets ratio', 'Debt ratio', 'Debt to equity ratio',
       'Equity multiplier'

In [20]:
ratios_df = merged3_df[['FYQ', 'Fiscal Year', 'Quarter', 
       'Reciprocal of times interest earned', 'Reciprocal of cash coverage'
       , 'Debt ratio', 'Debt to equity ratio', 'Equity multiplier', 'Long-term debt ratio',
        'Current ratio', 'Quick ratio', 'Cash ratio', 'NWC to assets ratio', 
         'Inventory turnover',
         'Receivables turnover',
         'Total asset turnover',
       'Net working capital turnover', 'Return on assets', 'Return on equity', 
        'Return on capital employed', 'Capital employed turnover',
    'PE ratio']]

In [21]:
ratios_df.to_csv('Ratios.csv')

In [22]:
days_df = merged3_df[['FYQ', 'Fiscal Year', 'Quarter', 
       'Days interval measure', 
       'Days sales in inventory', 
       'Days sales in receivable']]

In [23]:
days_df.to_csv('Days.csv')

In [24]:
market_df = merged3_df[['FYQ', 'Fiscal Year', 'Quarter', 
       'Price', 'EPS', 'PE ratio']]

In [25]:
market_df.to_csv('market.csv')